In [1]:
import xgboost as xgb
import numpy as np
import pandas as pd

In [2]:
cd 'drive'/'MyDrive'/'Dacon'/'LDI'

/content/drive/MyDrive/Dacon/LDI


In [3]:
train_err_data = pd.read_csv('./dataset/train_err_data.csv',thousands = ',')
train_quality_data = pd.read_csv('./dataset/train_quality_data.csv',thousands = ',')
train_problem_data = pd.read_csv('./dataset/train_problem_data.csv',thousands = ',')

In [4]:
df_tqd = train_quality_data.drop_duplicates()
df_ted = train_err_data.drop_duplicates()
df_tpd = train_problem_data.drop_duplicates()

In [5]:
df_tqd.columns

Index(['time', 'user_id', 'fwver', 'quality_0', 'quality_1', 'quality_2',
       'quality_3', 'quality_4', 'quality_5', 'quality_6', 'quality_7',
       'quality_8', 'quality_9', 'quality_10', 'quality_11', 'quality_12'],
      dtype='object')

In [6]:
df_ted.head()

,user_id,time,model_nm,fwver,errtype,errcode
0,10000,20201101025616,model_3,05.15.2138,15,1
1,10000,20201101030309,model_3,05.15.2138,12,1
2,10000,20201101030309,model_3,05.15.2138,11,1
3,10000,20201101050514,model_3,05.15.2138,16,1
4,10000,20201101050515,model_3,05.15.2138,4,0


In [7]:
df_ted.loc[(df_ted['errtype'] == 23) & (df_ted['errcode'] == 'connection timeout'),'errtype'] = '23_ct'
df_ted.loc[(df_ted['errtype'] == 23) & (df_ted['errcode'] == 'active'),'errtype'] = '23_as'
df_ted.loc[(df_ted['errtype'] == 23) & (df_ted['errcode'] == 'standby'),'errtype'] = '23_ac'
df_ted.loc[(df_ted['errtype'] == 31) & (df_ted['errcode'] == '0'),'errtype'] = '31_0'
df_ted.loc[(df_ted['errtype'] == 40) & (df_ted['errcode'] == '0'),'errtype'] = '40_0'

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [8]:
for i in range(0,13):
    qstr = 'quality_'+str(i)
    df_tqd.loc[df_tqd[qstr] > 0,qstr] = 1

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

In [9]:
df_ted = df_ted.drop('fwver',axis=1)
df_ted = df_ted.drop('model_nm',axis=1)
df_tqd = df_tqd.drop('fwver',axis=1)
df_ted = df_ted.drop('errcode',axis=1)

In [10]:
df_encoded_ted = pd.get_dummies(df_ted,columns=['errtype'])

In [11]:
times = list(df_ted['time'])

In [12]:
times = list(df_ted['time'])

for i in range(len(times)):
    try:
        times[i] = int(str(times[i])[4:8])
    except ValueError:
        print(i)
    

df_ted['time'] = times

df_ted.head()

,user_id,time,errtype
0,10000,1101,15
1,10000,1101,12
2,10000,1101,11
3,10000,1101,16
4,10000,1101,4


In [13]:
print(len(df_ted['time'].unique()))
print(len(df_ted['errtype'].unique()))

33
46


In [14]:
df_err = df_ted[['user_id','errtype']].values
df_err

array([[10000, 15],
       [10000, 12],
       [10000, 11],
       ...,
       [24999, 4],
       [24999, 4],
       [24999, 15]], dtype=object)

In [15]:
from tqdm import tqdm

df_x = np.zeros((15000,60))
df_err = df_ted[['user_id','errtype']].values
for pi,err in tqdm(df_err):
    if err == '23_ct':
        err = 43
    elif err == '23_as':
        err = 44
    elif err == '23_ac':
        err = 45
    elif err == '31_0':
        err = 46
    elif err == '40_0':
        err = 47
    df_x[int(pi)-10000,int(err)-1] += 1

for i in range(0,13):
    a = "quality_"+ str(i)
    df_qua = df_tqd[['user_id',a]].values
    for pi, qua in tqdm(df_qua):
        if qua == 1:
            df_x[int(pi)-10000,i + 47] += 1

df_err.shape

100%|██████████| 284193/284193 [00:00<00:00, 547584.52it/s]


(15368002, 2)

In [16]:
problem_list = list(df_tpd['user_id'].unique())
y = np.zeros(15000,)
for pi in problem_list:
    y[pi-10000] = 1

In [36]:
columns = []
for i in range(1,43):
    s = 'err_' + str(i)
    columns.append(s)
columns.append('23_ct')
columns.append('23_as')
columns.append('23_ac')
columns.append('31_0')
columns.append('40_0')
for i in range(13):
    s = 'quality_' + str(i)
    columns.append(s)
df = pd.DataFrame(df_x,columns=columns)
df.head()

,err_1,err_2,err_3,err_4,err_5,err_6,err_7,err_8,err_9,err_10,err_11,err_12,err_13,err_14,err_15,err_16,err_17,err_18,err_19,err_20,err_21,err_22,err_23,err_24,err_25,err_26,err_27,err_28,err_29,err_30,err_31,err_32,err_33,err_34,err_35,err_36,err_37,err_38,err_39,err_40,err_41,err_42,23_ct,23_as,23_ac,31_0,40_0,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
0,0.0,0.0,8.0,104.0,0.0,1.0,1.0,0.0,0.0,7.0,15.0,16.0,1.0,10.0,59.0,60.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,32.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0
1,0.0,0.0,0.0,0.0,48.0,1.0,1.0,0.0,0.0,0.0,10.0,11.0,1.0,8.0,143.0,128.0,0.0,4.0,1.0,3.0,1.0,756.0,0.0,5.0,1.0,22.0,0.0,0.0,0.0,0.0,124.0,0.0,10.0,18.0,0.0,1.0,1.0,0.0,0.0,57.0,56.0,1.0,0.0,126.0,625.0,126.0,56.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,2.0,131.0,1.0,2.0,1.0,0.0,0.0,1.0,13.0,14.0,1.0,4.0,52.0,52.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,25.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,12.0,3.0,6.0,0.0,2.0,20.0,0.0,0.0
3,0.0,0.0,0.0,0.0,2.0,1.0,1.0,0.0,0.0,0.0,9.0,9.0,0.0,0.0,45.0,30.0,1.0,0.0,0.0,0.0,0.0,28.0,0.0,0.0,0.0,57.0,0.0,0.0,0.0,0.0,32.0,0.0,8.0,0.0,0.0,1.0,1.0,2.0,0.0,9.0,1.0,0.0,7.0,12.0,0.0,33.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,3.0,4.0,0.0,0.0,0.0,16.0,19.0,3.0,5.0,143.0,91.0,0.0,0.0,0.0,0.0,0.0,140.0,3.0,0.0,0.0,33.0,0.0,0.0,0.0,0.0,81.0,0.0,16.0,0.0,0.0,1.0,1.0,0.0,0.0,2.0,0.0,2.0,104.0,7.0,5.0,95.0,2.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0,2.0,0.0,0.0,6.0,0.0,0.0


In [37]:
df.drop(['err_29','quality_3','quality_4'], axis = 1, inplace = True)

In [38]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
df_s = ss.fit_transform(df)
df= pd.DataFrame(df_s,columns=df.columns)
df.describe()

,err_1,err_2,err_3,err_4,err_5,err_6,err_7,err_8,err_9,err_10,err_11,err_12,err_13,err_14,err_15,err_16,err_17,err_18,err_19,err_20,err_21,err_22,err_23,err_24,err_25,err_26,err_27,err_28,err_30,err_31,err_32,err_33,err_34,err_35,err_36,err_37,err_38,err_39,err_40,err_41,err_42,23_ct,23_as,23_ac,31_0,40_0,quality_0,quality_1,quality_2,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
count,1.500000e+04,1.500000e+04,1.500000e+04,1.500000e+04,1.500000e+04,1.500000e+04,1.500000e+04,1.500000e+04,1.500000e+04,1.500000e+04,1.500000e+04,1.500000e+04,1.500000e+04,1.500000e+04,1.500000e+04,1.500000e+04,1.500000e+04,1.500000e+04,1.500000e+04,1.500000e+04,1.500000e+04,1.500000e+04,1.500000e+04,1.500000e+04,1.500000e+04,1.500000e+04,1.500000e+04,1.500000e+04,1.500000e+04,1.500000e+04,1.500000e+04,1.500000e+04,1.500000e+04,1.500000e+04,1.500000e+04,1.500000e+04,1.500000e+04,1.500000e+04,1.500000e+04,1.500000e+04,1.500000e+04,1.500000e+04,1.500000e+04,1.500000e+04,1.500000e+04,1.500000e+04,1.500000e+04,1.500000e+04,1.500000e+04,1.500000e+04,1.500000e+04,1.500000e+04,1.500000e+04,1.500000e+04,1.500000e+04,1.500000e+04,1.500000e+04
mean,1.030498e-15,2.016223e-15,6.301325e-17,1.246142e-16,-2.511269e-16,-1.906207e-16,-5.770414e-16,8.088928e-16,-1.017107e-15,-6.312913e-16,-2.592001e-16,-1.001939e-16,9.439856e-17,-4.443668e-16,-4.136691e-17,3.648933e-18,-2.053469e-16,8.246589e-16,-7.868687e-16,-7.911449e-17,5.863402e-16,2.164750e-17,-2.669716e-16,-2.775650e-16,4.889903e-16,-1.418199e-16,-7.334975e-16,-8.212412e-17,-8.974710e-16,7.178554e-16,-9.262332e-16,-5.000296e-16,-2.626677e-16,-1.675785e-15,6.082394e-16,3.979335e-16,-9.296267e-18,-4.476419e-16,4.645913e-17,2.703319e-16,-6.997958e-16,3.420523e-16,5.430018e-16,2.520650e-16,5.469144e-16,5.409747e-17,7.737394e-16,5.056381e-16,-9.643115e-16,-1.504685e-16,-9.669617e-16,-3.774666e-16,-5.600002e-16,5.613473e-16,4.269696e-16,-1.332342e-15,7.667866e-16
std,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00
min,-1.968519e-01,-3.011846e-02,-2.904574e-02,-4.578863e-02,-2.960069e-01,-1.074770e-01,-1.032760e-01,-5.236141e-02,-3.766889e-02,-4.869290e-02,-2.151626e-01,-2.227535e-01,-6.473408e-02,-6.976125e-02,-1.504373e+00,-1.292973e+00,-2.797238e-01,-2.251618e-01,-1.140276e-01,-2.526766e-01,-1.082436e-01,-2.255527e-01,-1.481161e-01,-7.195615e-02,-7.518133e-02,-1.559604e+00,-4.594438e-02,-4.528162e-02,-1.545195e-01,-3.805843e-01,-1.989563e-01,-1.052305e+00,-6.154607e-02,-7.415870e-02,-1.030661e+00,-1.026064e+00,-1.143985e-01,-8.013076e-02,-6.097187e-01,-3.368412e-01,-6.443708e-01,-1.702823e-01,-1.178268e-01,-1.703549e-01,-3.824948e-01,-5.944277e-01,-2.577372e-02,-1.231241e-01,-2.832941e-02,-1.510986e-01,-8.292263e-02,-1.026440e-01,-1.258090e-01,-3.214412e-02,-1.871966e-01,-6.567623e-02,-7.020136e-02
25%,-1.968519e-01,-3.011846e-02,-2.904574e-02,-4.578863e-02,-2.960069e-01,-1.074770e-01,-1.032760e-01,-5.236141e-02,-3.766889e-02,-4.869290e-02,-7.769202e-02,-7.604290e-02,-6.473408e-02,-6.976125e-02,-6.523056e-01,-6.403444e-01,-2.797238e-01,-2.251618e-01,-1.140276e-01,-2.526766e-01,-1.082436e-01,-2.255527e-01,-1.481161e-01,-7.195615e-02,-7.518133e-02,-5.903548e-01,-4.594438e-02,-4.528162e-02,-1.545195e-01,-3.805843e-01,-1.989563e-01,-1.052305e+00,-6.154607e-02,-7.415870e-02,-1.030661e+00,-1.026064e+00,-

In [39]:
df.drop(['err_9','err_10','err_16'], axis = 1, inplace = True)

In [40]:
df.drop(['23_ct','err_18','err_19','err_21','quality_11','quality_6','err_28'], axis = 1, inplace = True)

In [41]:
df.drop(['quality_2'], axis = 1, inplace = True)

In [42]:
df.drop(['quality_0'], axis = 1, inplace = True)

In [43]:
df.drop(['err_6'], axis = 1, inplace = True)

In [44]:
df.drop(['err_11','err_12','err_35'], axis = 1, inplace = True)

In [45]:
df.drop(['quality_10','err_17'], axis = 1, inplace = True)

In [46]:
df.drop(['err_36','quality_8','31_0','40_0'], axis = 1, inplace = True)

In [47]:
corr = df.corr(method='pearson')
display(corr)

,err_1,err_2,err_3,err_4,err_5,err_7,err_8,err_13,err_14,err_15,err_20,err_22,err_23,err_24,err_25,err_26,err_27,err_30,err_31,err_32,err_33,err_34,err_37,err_38,err_39,err_40,err_41,err_42,23_as,23_ac,quality_1,quality_5,quality_7,quality_9,quality_12
err_1,1.000000,-0.005914,0.000468,0.035891,-0.054580,-0.000274,0.031853,-0.006501,-0.006441,-0.123566,-0.048239,-0.043459,-0.028572,-0.013561,-0.014373,-0.299145,0.182765,-0.011425,-0.073634,-0.038347,-0.202771,-0.011916,-0.197351,-0.021080,-0.015391,-0.118306,-0.065070,-0.124581,-0.022543,-0.032608,0.043307,0.012922,0.005029,0.012719,0.207331
err_2,-0.005914,1.000000,-0.000790,-0.001164,-0.001861,0.003275,-0.001542,-0.000591,-0.001341,0.000954,0.005589,-0.005358,-0.004093,-0.001803,0.005219,-0.017387,-0.001368,0.005100,0.000725,-0.004052,0.003258,-0.000528,0.015210,-0.000457,-0.001341,0.001389,0.003282,-0.000290,-0.002165,-0.000852,-0.003407,0.015417,-0.001536,-0.000968,-0.002033
err_3,0.000468,-0.000790,1.000000,0.002924,-0.007192,0.029399,-0.001149,0.013094,0.317312,-0.010053,-0.007223,-0.006533,-0.004299,-0.000385,-0.002167,-0.008756,-0.000341,-0.002989,-0.010991,-0.005756,-0.030069,-0.001787,-0.029274,-0.003230,-0.002318,-0.017560,-0.009710,-0.018635,-0.003411,-0.004910,0.013400,0.038883,0.020425,0.077467,0.002236
err_4,0.035891,-0.001164,0.002924,1.000000,-0.011763,0.004684,0.010485,0.000645,0.000442,-0.011021,-0.010526,-0.010174,-0.006669,-0.001271,-0.003335,-0.022528,0.202155,-0.004640,-0.017201,-0.008972,-0.047680,-0.002789,-0.045746,-0.005063,-0.003621,-0.027567,-0.015251,-0.029137,-0.005356,-0.007717,0.002890,0.020528,0.003509,0.007549,0.036662
err_5,-0.054580,-0.001861,-0.007192,-0.011763,1.000000,-0.000731,0.003506,-0.004244,0.003932,0.098226,0.077890,0.044616,0.037589,-0.002322,0.028980,0.111628,-0.012191,0.019062,0.049570,0.047689,0.092087,0.019738,0.097132,0.028140,0.025502,0.121230,0.080372,0.036813,0.022057,0.010764,0.038873,-0.004586,-0.002627,-0.008515,-0.009731
err_7,-0.000274,0.003275,0.029399,0.004684,-0.000731,1.000000,-0.001710,0.180098,0.260709,0.001638,-0.001042,-0.000101,-0.000657,0.005884,-0.003996,0.002364,0.026084,0.038578,0.000871,0.006713,-0.004996,0.002908,-0.004208,0.121205,0.000626,0.003125,0.005200,0.001948,0.003050,-0.003039,0.143204,0.085155,0.416499,0.007829,0.016104
err_8,0.031853,-0.001542,-0.001149,0.010485,0.003506,-0.001710,1.000000,-0.002187,-0.002783,-0.012770,-0.002292,-0.004718,-0.004188,0.000093,-0.002348,-0.021626,0.015286,-0.006168,-0.006184,-0.006864,-0.018660,0.003430,-0.004793,-0.004603,-0.002751,-0.006260,0.007970,-0.002065,0.000427,-0.002335,-0.003383,0.004172,-0.001652,-0.001087,-0.002869
err_13,-0.006501,-0.000591,0.013094,0.000645,-0.004244,0.180098,-0.002187,1.000000,0.052454,-0.012607,-0.001207,-0.003729,-0.002221,0.013548,-0.001505,-0.007052,-0.001316,0.001142,-0.004516,-0.003012,-0.005481,-0.000225,-0.013243,0.009901,-0.002440,-0.006283,0.000030,-0.010066,-0.002639,-0.001541,0.013910,0.007854,0.052225,0.009069,0.000943
err_14,-0.006441,-0.001341,0.317312,0.000442,0.003932,0.260709,-0.002783,0.052454,1.000000,-0.008487,-0.007935,-0.007598,-0.005834,-0.001524,-0.003113,-0.004507,0.002388,0.012209,-0.004483,0.001413,-0.013299,0.002435,-0.009452,0.050881,-0.003376,-0.009741,-0.009824,-0.009718,-0.004291,-0.005176,0.070377,0.052596,0.124456,0.024918,0.005814
err_15,-0.123566,0.000954,-0.010053,-0.011021,0.098226,0.001638,-0.012770,-0.012607,-0.008487,1.000000,0.134282,0.068750,0.046018,0.007459,0.046736,0.415323,-0.029628,-0.022929,0.233559,0.045633,0.256269,0.026902,0.279452,0.021230,0.047748,0.516844,0.187855,0.230661,0.053151,0.072102,0.003892,0.006465,-0.024345,-0.008715,0.006665


In [48]:
df.shape

(15000, 35)

In [49]:
import pickle
with open('./Preprocessed_data/v4/traindata.txt', 'wb') as f:
    pickle.dump(df, f)